In [ ]:
import dcarte
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.gridspec import GridSpec

import pyarrow as pa
import pyarrow.parquet as pq

import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
## read mmse
mmse = pd.read_csv('redcap_16_nov_2023/mmse_MinderHealthManageme_DATA_LABELS_2023-11-16_1135.csv')

mmse = mmse.rename(columns={'Participants ID':'DYAD','Date of testing':'date_test_mmse','Total Score':'tot_score_mmse'})
mmse = mmse[['DYAD','date_test_mmse','tot_score_mmse']]
mmse['DYAD'] = mmse['DYAD'].ffill()
mmse = mmse[['DYAD','date_test_mmse','tot_score_mmse']]
mmse = mmse.dropna().reset_index(drop=True)
mmse 

## read dyad
maps= pd.read_excel('dyads_unprotected.xlsx')
maps['id'] = maps['Research ID'].apply(lambda x:x[:5])
maps['DYAD'] = maps['DYAD'].apply(lambda x:str(x))

## merge
merge1 = mmse.merge(maps, on='DYAD',how='left').dropna()
merge1

In [ ]:
sorted_ids = pd.read_csv('sorted_ids_14_11_2023.csv')

## here by dropping nulls I limit to the participants in my data
merge1 = merge1[['id','date_test_mmse','tot_score_mmse']].merge(sorted_ids[['participant_id','min_date', 'max_date']], 
                                                                 left_on='id', right_on='participant_id', how='left').dropna()
merge1.head(3)

In [ ]:
#number of participants
len(merge1['id'].unique()) #67

In [ ]:
# days of data
len(merge1) #127

In [ ]:
merge1['days_after_min_date'] = (pd.to_datetime(merge1['date_test_mmse'],dayfirst=True)-pd.to_datetime(merge1['min_date'],dayfirst=True)).dt.days
merge1['days_after_max_date'] = (pd.to_datetime(merge1['date_test_mmse'],dayfirst=True)-pd.to_datetime(merge1['max_date'],dayfirst=True)).dt.days
merge1.head(7)

In [ ]:
# days before min date

print(len(merge1[merge1['days_after_min_date']<0]), len(merge1[merge1['days_after_min_date']<0]['id'].unique()),
      merge1[merge1['days_after_min_date']<0]['days_after_min_date'].mean())

# days after max date
print(len(merge1[merge1['days_after_max_date']>0]), len(merge1[merge1['days_after_max_date']>0]['id'].unique()),
      merge1[merge1['days_after_max_date']>0]['days_after_max_date'].mean())

# # drop scores after max date
# merge1 = merge1[merge1['days_after_max_date']<=0].reset_index(drop=True)  #is positive after max date
# #drop scores before min date
# merge1 = merge1[merge1['days_after_min_date']>=0].reset_index(drop=True)  #is negative before min date
# # days of data
# print(len(merge1))
# #number of participants
# print(len(merge1['id'].unique()))

In [ ]:
merge1[['id','max_date']].groupby(by=['id']).count().reset_index().groupby(by=['max_date']).count()

In [ ]:
merge1['closest_to_initial_day']=False
merge1['absolute_days_after_min_date'] = abs(merge1['days_after_min_date'])
merge1.loc[merge1.groupby('id', sort=False)['absolute_days_after_min_date'].idxmin(),'closest_to_initial_day']=True
merge1 = merge1[['id', 'date_test_mmse', 'tot_score_mmse', 'participant_id', 'min_date',
       'max_date', 'days_after_min_date', 'days_after_max_date',
       'closest_to_initial_day']]
merge1 

In [ ]:
ids = np.load('ids_seven_days_each_five_bimonth.npy',allow_pickle=True)
len(ids)

# participant without mmse
list(set(ids) - set(merge1['id'].unique()))

In [ ]:
len(merge1[merge1['closest_to_initial_day']==True]['id'].unique())

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(merge1[merge1['closest_to_initial_day']==True]['tot_score_mmse'], bins=range(0,32,1),rwidth=0.9, color='tomato')


plt.yticks([1,3,5,7,9], rotation=0)
plt.xticks([0.5,5.5,10.5,15.5,20.5,25.5,30.5],[0,5,10,15,20,25,30], rotation=0)
plt.xlabel('MMSE Score Per Participant')
plt.show()

In [ ]:
merge1.columns

In [ ]:
merge1[['participant_id','closest_to_initial_day', 'date_test_mmse', 'tot_score_mmse','min_date',
      'max_date', 'days_after_min_date', 'days_after_max_date']].to_excel("redcap_mmse_baseline_values_nov30.xlsx")